In [1]:
import torch
import gpytorch

from scalable_gp_inference.hparam_training import train_exact_gp

In [2]:
torch.set_default_dtype(torch.float64)
torch.manual_seed(0)

In [3]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu"
device = torch.device("cuda:1")
n = 10000
d = 3

# Training data is n points in [0,1] inclusive regularly spaced
train_x = torch.linspace(0, 1, n).unsqueeze(1).expand(-1, d)
# True function is sin(2*pi*x) with Gaussian noise
freqs = 2 * torch.pi * torch.randn(d)
train_y = torch.sin(train_x @ freqs) + \
    torch.randn(train_x.shape[0]) * (0.04 ** 0.5)

train_x = train_x.to(device)
train_y = train_y.to(device)

In [4]:
hparams = train_exact_gp(train_x, train_y, "rbf", {"lr": 0.1}, 100)
print(hparams.signal_variance)
print(hparams.kernel_lengthscale)
print(hparams.noise_variance)

1.3410016050517224
tensor([[0.6043, 0.6043, 0.6043]], device='cuda:1')
0.041294909693002424


In [5]:
hparams_doubled = hparams + hparams
print(hparams_doubled.signal_variance)
print(hparams_doubled.kernel_lengthscale)
print(hparams_doubled.noise_variance)

2.682003210103445
tensor([[1.2086, 1.2086, 1.2086]], device='cuda:1')
0.08258981938600485


In [6]:
print(hparams / 3)
print(hparams / 3.0)

GPHparams(signal_variance=0.4470005350172408, kernel_lengthscale=tensor([[0.2014, 0.2014, 0.2014]], device='cuda:1'), noise_variance=0.013764969897667475)
GPHparams(signal_variance=0.4470005350172408, kernel_lengthscale=tensor([[0.2014, 0.2014, 0.2014]], device='cuda:1'), noise_variance=0.013764969897667475)


In [7]:
# class ExactGPModel(gpytorch.models.ExactGP):
#     def __init__(self, train_x, train_y, likelihood, base_kernel):
#         super().__init__(train_x, train_y, likelihood)
#         self.mean_module = gpytorch.means.ConstantMean()
#         self.covar_module = gpytorch.kernels.ScaleKernel(base_kernel)

#     def forward(self, x):
#         mean_x = self.mean_module(x)
#         covar_x = self.covar_module(x)
#         return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [8]:
# # initialize likelihood and model
# likelihood = gpytorch.likelihoods.GaussianLikelihood()
# # base_kernel = gpytorch.kernels.RBFKernel(ard_num_dims=d)
# base_kernel = gpytorch.kernels.MaternKernel(nu=2.5, ard_num_dims=d)
# model = ExactGPModel(train_x, train_y, likelihood, base_kernel)
# model = model.to(device)
# likelihood = likelihood.to(device)

In [9]:
# # Find optimal model hyperparameters
# model.train()
# likelihood.train()

# # Use the Adam optimizer
# # Includes GaussianLikelihood parameters
# optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

# # "Loss" for GPs - the marginal log likelihood
# mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# training_iter = 100
# for i in range(training_iter):
#     # Zero gradients from previous iteration
#     optimizer.zero_grad()
#     # Output from model
#     output = model(train_x)
#     # Calc loss and backprop gradients
#     loss = -mll(output, train_y)
#     loss.backward()
#     print(f'Iter {i+1}/{training_iter} - Loss: {loss.item():.3f}   signal: {model.covar_module.outputscale.item():.3f}   '
#         f'lengthscale: {model.covar_module.base_kernel.lengthscale.detach().cpu().numpy()}   '
#         f'noise: {model.likelihood.noise.item():.3f}')
#     optimizer.step()